# Trialing the Light Gradient Boost model
The code will run the logistic regression model on the FTSE 350 stock data and see what the profit would have been from investing with the below criteria:
- Starting with £10,000
- Never investing more than 10% of the value into a single stock
- Tracking balance so once the account is empty no more can be invested until shares are liqidated
- Shares are bought at the open of the day following the first buy signal, if shares are not already held
- Shares are sold at the open of the day following the first close signal after a hold period, if shares are held

Trading variables:
- Trades cost £2.50 to execute
- Spread is 1%

In [1]:
#Import models
import numpy as np
import pandas as pd
import math
import lightgbm as lgb
from sklearn.externals import joblib as jl
import os
import tables
from rf_modules import *

C:\Users\Robert\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
#Import and combine prices files
path = r'C:\Users\Robert\Documents\python_scripts\stock_trading_ml_modelling\historical_prices'
df_ft = pd.read_hdf(path +r'\all_hist_prices_w_ft_eng2.h5')
print('SHAPE: {}'.format(df_ft.shape))
print(df_ft.columns)
df_ft.head()

SHAPE: (281287, 144)
Index(['ticker', 'date', 'open', 'close', 'high', 'low', 'volume',
       'change_price', 'per_change_price', 'ema26',
       ...
       'prev_min_grad_volume', 'prev_grad_conv_volume', 'max_move_cum_volume',
       'min_move_cum_volume', 'long_prev_max_move_date_volume',
       'long_prev_min_move_date_volume', 'long_max_grad_volume',
       'long_min_grad_volume', 'long_grad_conv_volume', 'signal'],
      dtype='object', length=144)


,ticker,date,open,close,high,low,volume,change_price,per_change_price,ema26,...,prev_min_grad_volume,prev_grad_conv_volume,max_move_cum_volume,min_move_cum_volume,long_prev_max_move_date_volume,long_prev_min_move_date_volume,long_max_grad_volume,long_min_grad_volume,long_grad_conv_volume,signal
214063,SBRY,1995-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0.0,0.0,NaN,NaN,NaN,hold
214064,SBRY,1995-01-09,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,NaN,...,NaN,NaN,0,0,0.0,0.0,NaN,NaN,NaN,hold
214065,SBRY,1995-01-16,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,NaN,...,NaN,NaN,0,0,0.0,0.0,NaN,NaN,NaN,hold
214066,SBRY,1995-01-23,0.125278,1.000000,1.000000,0.599644,0.861339,0.874722,0.874722,NaN,...,NaN,NaN,0,0,0.0,0.0,NaN,NaN,NaN,sell
214067,SBRY,1995-01-30,1.000000,0.833416,0.861097,1.000000,0.546860,-0.166584,-0.199881,NaN,...,NaN,NaN,0,0,0.0,0.0,NaN,NaN,NaN,hold


In [3]:
#Import to lr model
lgb_mod = jl.load(path+r'\lgb_model.joblib')
lgb_mod

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=22,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=848, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [4]:
print('SHAPE BEFORE: {}'.format(df_ft.shape))
# data_df = df_ft.replace([np.inf,-np.inf],np.nan).dropna(axis=0)
data_df = df_ft
print('SHAPE AFTER: {}'.format(data_df.shape))
data_df.head()

SHAPE BEFORE: (281287, 144)
SHAPE AFTER: (281287, 144)


,ticker,date,open,close,high,low,volume,change_price,per_change_price,ema26,...,prev_min_grad_volume,prev_grad_conv_volume,max_move_cum_volume,min_move_cum_volume,long_prev_max_move_date_volume,long_prev_min_move_date_volume,long_max_grad_volume,long_min_grad_volume,long_grad_conv_volume,signal
214063,SBRY,1995-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0.0,0.0,NaN,NaN,NaN,hold
214064,SBRY,1995-01-09,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,NaN,...,NaN,NaN,0,0,0.0,0.0,NaN,NaN,NaN,hold
214065,SBRY,1995-01-16,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,NaN,...,NaN,NaN,0,0,0.0,0.0,NaN,NaN,NaN,hold
214066,SBRY,1995-01-23,0.125278,1.000000,1.000000,0.599644,0.861339,0.874722,0.874722,NaN,...,NaN,NaN,0,0,0.0,0.0,NaN,NaN,NaN,sell
214067,SBRY,1995-01-30,1.000000,0.833416,0.861097,1.000000,0.546860,-0.166584,-0.199881,NaN,...,NaN,NaN,0,0,0.0,0.0,NaN,NaN,NaN,hold


In [5]:
#Import feature_list
f = open(path + r'\lgb_model_feature_list.txt','r')
feature_li = f.read().split(',')
feature_li

['open',
 'close',
 'high',
 'low',
 'volume',
 'change_price',
 'per_change_price',
 'ema26',
 'macd',
 'signal_line',
 'macd_line',
 'close_orig_per_change_max_4',
 'close_orig_per_change_max_13',
 'close_orig_per_change_max_26',
 'close_orig_per_change_max_52',
 'close_orig_per_change_min_4',
 'close_orig_per_change_min_13',
 'close_orig_per_change_min_26',
 'close_orig_per_change_min_52',
 'macd_line_per_change_max_4',
 'macd_line_per_change_max_13',
 'macd_line_per_change_max_26',
 'macd_line_per_change_max_52',
 'macd_line_per_change_min_4',
 'macd_line_per_change_min_13',
 'macd_line_per_change_min_26',
 'macd_line_per_change_min_52',
 'change_close_shift1',
 'close_max',
 'close_min',
 'prev_max_close',
 'prev_min_close',
 'prev_max_close_date_change',
 'prev_min_close_date_change',
 'max_change_close',
 'min_change_close',
 'prev_max_grad_close',
 'prev_min_grad_close',
 'prev_grad_conv_close',
 'max_move_cum_close',
 'min_move_cum_close',
 'long_prev_max_move_date_close',
 'l

In [6]:
#Run the rf_mod to get signals
data_df['signal'] = lgb_mod.predict(data_df[feature_li])
data_df['signal_prob'] = [x.max() for x in lgb_mod.predict_proba(data_df[feature_li])]

In [7]:
#Show current buy ratings
print('BUY COUNT: {:,}'.format(data_df.loc[(data_df['date'] == data_df['date'].max()) & (data_df['signal'] == 'buy')].shape[0]))
data_df.loc[(data_df['date'] == data_df['date'].max()) & (data_df['signal'] == 'buy'),['ticker','signal','signal_prob']].sort_values(['signal','signal_prob'],ascending=[True,False])

BUY COUNT: 0


,ticker,signal,signal_prob


In [8]:
#Show current sell ratings
print('SELL COUNT: {:,}'.format(data_df.loc[(data_df['date'] == data_df['date'].max()) & (data_df['signal'] == 'sell')].shape[0]))
data_df.loc[(data_df['date'] == data_df['date'].max()) & (data_df['signal'] == 'sell'),['ticker','signal','signal_prob']].sort_values(['signal','signal_prob'],ascending=[True,False])

SELL COUNT: 22


,ticker,signal,signal_prob
73523,DPLM,sell,0.648260
172627,OSB,sell,0.645287
214062,SBRE,sell,0.640893
249404,SXS,sell,0.630015
70042,DNLM,sell,0.609584
61853,CSP,sell,0.603314
123935,HWDN,sell,0.600680
186932,PNN,sell,0.598514
195077,QQ,sell,0.597291
5664,AGR,sell,0.584382


In [9]:
#Show current hold ratings
print('HOLD COUNT: {:,}'.format(data_df.loc[(data_df['date'] == data_df['date'].max()) & (data_df['signal'] == 'hold')].shape[0]))
data_df.loc[(data_df['date'] == data_df['date'].max()) & (data_df['signal'] == 'hold'),['ticker','signal','signal_prob']].sort_values(['signal','signal_prob'],ascending=[True,False])

HOLD COUNT: 279


,ticker,signal,signal_prob
210167,RSE,hold,0.920692
155663,MCRO,hold,0.892272
130167,IMB,hold,0.882471
202446,RHIM,hold,0.851504
266839,UTG,hold,0.800460
174096,OXIG,hold,0.790349
192559,PSON,hold,0.789161
166848,NETW,hold,0.787896
46781,CCL,hold,0.782287
43080,CARD,hold,0.777869


# Combine with price data and create ledger

In [10]:
#Import and combine prices files
df_prices = pd.read_hdf(path +r'\all_hist_prices_w.h5')

In [11]:
#Sort by ticker and date then add the open_shift_neg1 field
#These allow the buying and selling to be done at a realistic price
df_prices.sort_values(['ticker','date'],ascending=[True,True],inplace=True)
df_prices['open_shift_neg1'] = df_prices['open'].shift(-1)
df_prices['date'] = df_prices['date'].astype('datetime64')
print('SHAPE: {}'.format(df_prices.shape))
print(df_prices.columns)
df_prices.iloc[90:95]

SHAPE: (280168, 14)
Index(['ticker', 'date', 'high', 'low', 'volume', 'open', 'close', 'change',
       'ema12', 'ema26', 'macd_line', 'signal_line', 'macd',
       'open_shift_neg1'],
      dtype='object')


,ticker,date,high,low,volume,open,close,change,ema12,ema26,macd_line,signal_line,macd,open_shift_neg1
90,3IN,2009-09-21,142.85,137.15,7932371.0,139.26,142.57,3.31,136.971580,134.290336,2.681244,1.988463,0.692781,140.39
91,3IN,2009-09-28,145.67,139.62,5390930.0,140.39,144.96,4.57,138.200568,135.080682,3.119886,2.214748,0.905138,144.26
92,3IN,2009-10-05,146.09,142.99,3520386.0,144.26,144.68,0.42,139.197403,135.791742,3.405661,2.452930,0.952731,143.84
93,3IN,2009-10-12,146.37,142.99,2676531.0,143.84,144.82,0.98,140.062418,136.460502,3.601916,2.682727,0.919189,146.37
94,3IN,2009-10-19,147.07,142.29,3350132.0,146.37,143.56,-2.81,140.600508,136.986391,3.614117,2.869005,0.745112,144.26


In [12]:
#Join on the buy and sell signals
df_prices = pd.merge(df_prices,data_df[['ticker','date','signal','signal_prob']],left_on=['ticker','date'],right_on=['ticker','date'],how='inner')
print('SHAPE: {}'.format(df_prices.shape))
print(df_prices.columns)
print(df_prices.signal.value_counts())
df_prices.head()

SHAPE: (287263, 16)
Index(['ticker', 'date', 'high', 'low', 'volume', 'open', 'close', 'change',
       'ema12', 'ema26', 'macd_line', 'signal_line', 'macd', 'open_shift_neg1',
       'signal', 'signal_prob'],
      dtype='object')
hold    201275
sell     47157
buy      38831
Name: signal, dtype: int64


,ticker,date,high,low,volume,open,close,change,ema12,ema26,macd_line,signal_line,macd,open_shift_neg1,signal,signal_prob
0,3IN,2007-12-31,149.89,147.07,1373801.0,149.18,147.43,-1.75,NaN,NaN,NaN,NaN,NaN,147.07,hold,0.999804
1,3IN,2008-01-07,149.54,147.07,2345191.0,147.07,148.83,1.76,NaN,NaN,NaN,NaN,NaN,148.48,hold,0.677251
2,3IN,2008-01-14,150.59,147.43,2150049.0,148.48,149.18,0.70,NaN,NaN,NaN,NaN,NaN,147.78,hold,0.792586
3,3IN,2008-01-21,154.82,145.32,3070968.0,147.78,152.00,4.22,NaN,NaN,NaN,NaN,NaN,149.89,hold,0.825994
4,3IN,2008-01-28,154.82,148.83,2510972.0,149.89,152.00,2.11,NaN,NaN,NaN,NaN,NaN,154.46,hold,0.734468


In [13]:
#Limit to a test period
df_prices = df_prices[df_prices['date'] >= '2014-01-01']
print('SHAPE: {}'.format(df_prices.shape))

SHAPE: (94306, 16)


In [14]:
#Create a dictionary of max character lengths of fields for use later in h5 file appending
def get_col_lens(_df_in):
    _col_lens = {}
    for c in _df_in:
        _tmp_s = pd.Series([len(str(x)) for x in _df_in[c]])
        _col_lens[c] = _tmp_s.max()
    return _col_lens
col_lens = get_col_lens(df_prices)
col_lens

{'ticker': 4,
 'date': 19,
 'high': 8,
 'low': 8,
 'volume': 12,
 'open': 7,
 'close': 7,
 'change': 21,
 'ema12': 18,
 'ema26': 18,
 'macd_line': 23,
 'signal_line': 23,
 'macd': 23,
 'open_shift_neg1': 8,
 'signal': 4,
 'signal_prob': 19}

In [15]:
#Write df_prices to a .h5 file
hf_store_name = path + r'\historic_lgb_bsh_signals_TMP.h5'
hf = pd.HDFStore(hf_store_name)
group_name = r'bsh_signals'
df_prices.to_hdf(hf_store_name,key=group_name,append=True,min_itemsize=col_lens)
hf.close()

In [16]:
#close any open h5 files
tables.file._open_files.close_all()

In [17]:
#Delete the old h5 file and rename the TMP
try:
    os.remove(path + r'\historic_lgb_bsh_signals.h5')
    print('\nSUCCESSFULLY REMOVED {}'.format(path + r'\historic_lgb_bsh_signals.h5'))
except Exception as e:
    print('\nERROR - REMOVING:{}'.format(e))
try:
    os.rename(path + r'\historic_lgb_bsh_signals_TMP.h5',path + r'\historic_lgb_bsh_signals.h5')
    print('\nSUCCESSFULLY RENAMED {} TO {}'.format(path + r'\historic_lgb_bsh_signals_TMP.h5',path + r'\historic_lgb_bsh_signals.h5'))
except Exception as e:
    print('\nERROR - RENAMING:{}'.format(e))


SUCCESSFULLY REMOVED C:\Users\Robert\Documents\python_scripts\stock_trading_ml_modelling\historical_prices\historic_lgb_bsh_signals.h5

SUCCESSFULLY RENAMED C:\Users\Robert\Documents\python_scripts\stock_trading_ml_modelling\historical_prices\historic_lgb_bsh_signals_TMP.h5 TO C:\Users\Robert\Documents\python_scripts\stock_trading_ml_modelling\historical_prices\historic_lgb_bsh_signals.h5


# Getting the status of a specific ticker

In [18]:
def get_signal():
    tick = input('\nEnter the ticker: ').upper()
    if tick in df_prices.ticker.unique():
        print('{} CURRENT STATUS -> {} - {}'.format(tick.upper(),df_prices.loc[(df_prices['date'] == df_prices['date'].max()) & (df_prices['ticker'] == tick.upper())].signal.values[0],df_prices['date'].max()))
        display(df_prices.loc[df_prices['ticker'] == tick.upper(),['ticker','date','signal','close']].sort_values(['date'],ascending=False))
        return None
    elif tick == 'EXIT':
        return None
    else:
        print('THERE IS NO SIGNAL FOR {}'.format(tick.upper()))
        get_signal()
get_signal()


Enter the ticker: sbry
SBRY CURRENT STATUS -> hold - 2019-12-09 00:00:00


,ticker,date,signal,close
222622,SBRY,2019-12-09,hold,223.2
222621,SBRY,2019-12-09,hold,223.2
222620,SBRY,2019-12-02,hold,220.0
222619,SBRY,2019-12-02,hold,220.0
222618,SBRY,2019-11-25,hold,214.3
222617,SBRY,2019-11-25,hold,214.3
222616,SBRY,2019-11-18,hold,214.4
222615,SBRY,2019-11-18,hold,214.4
222614,SBRY,2019-11-11,hold,204.8
222613,SBRY,2019-11-11,hold,204.8
